In [1]:
import pandas as pd
import numpy as np
import datetime
import numpy as np
import os
import csv
import cv2
import pandas as pd
import Gaussian
import Parameters
import Util
import matplotlib.pyplot as plt
from Dependency import Dependency

In [2]:
d = Dependency()
path = os.getcwd()[:-5] + 'Model Dependencies/'
read_path = path + 'Region_Mobility_Report_CSVs/'
path_2020 = read_path + '2020_CA_Region_Mobility_Report.csv'
path_2021 = read_path + '2021_CA_Region_Mobility_Report.csv'
path_2022 = read_path + '2022_CA_Region_Mobility_Report.csv'
df1 = pd.read_csv(path_2020)
df2 = pd.read_csv(path_2021)
df3 = pd.read_csv(path_2022)

KeyError: 'Algoma Public Health'

In [ ]:
df = pd.concat([df1, df2, df3])

In [ ]:
print(df)

In [ ]:
groups = df.groupby('iso_3166_2_code')
Ontario = groups.get_group('CA-ON')
Ontario['date'] = pd.to_datetime(Ontario['date'])
max_date = (Ontario['date'].max() - Parameters.OUTBREAK_FIRST_DAY).days
min_date = (Ontario['date'].min() - Parameters.OUTBREAK_FIRST_DAY).days
print(max_date, min_date)

In [ ]:
mobility = np.ones(shape=(6, d.total_days))
blurred_mobility = np.ones(shape=(6, d.total_days))

In [ ]:
retail = Ontario['retail_and_recreation_percent_change_from_baseline'].to_numpy()
grocery = Ontario['grocery_and_pharmacy_percent_change_from_baseline'].to_numpy()
park = Ontario['parks_percent_change_from_baseline'].to_numpy()
trainsit = Ontario['transit_stations_percent_change_from_baseline'].to_numpy()
workplace = Ontario['workplaces_percent_change_from_baseline'].to_numpy()
residential = Ontario['residential_percent_change_from_baseline'].to_numpy()
mobility[0][min_date:max_date+1] = retail
mobility[1][min_date:max_date+1] = grocery
mobility[2][min_date:max_date+1] = park
mobility[3][min_date:max_date+1] = trainsit
mobility[4][min_date:max_date+1] = workplace
mobility[5][min_date:max_date+1] = residential
mobility = mobility.T

In [ ]:
retail_blurred = cv2.GaussianBlur(retail.reshape(retail.shape[0], 1), (15, 15), 0)
grocery_blurred = cv2.GaussianBlur(grocery.reshape(grocery.shape[0], 1), (15, 15), 0)
park_blurred = cv2.GaussianBlur(park.reshape(park.shape[0], 1), (15, 15), 0)
trainsit_blurred = cv2.GaussianBlur(trainsit.reshape(trainsit.shape[0], 1), (15, 15), 0)
workplace_blurred = cv2.GaussianBlur(workplace.reshape(workplace.shape[0], 1), (15, 15), 0)
residential_blurred = cv2.GaussianBlur(residential.reshape(residential.shape[0], 1), (15, 15), 0)
blurred_mobility[0][min_date:max_date+1] = retail_blurred.flatten()
blurred_mobility[1][min_date:max_date+1] = grocery_blurred.flatten()
blurred_mobility[2][min_date:max_date+1] = park_blurred.flatten()
blurred_mobility[3][min_date:max_date+1] = trainsit_blurred.flatten()
blurred_mobility[4][min_date:max_date+1] = workplace_blurred.flatten()
blurred_mobility[5][min_date:max_date+1] = residential_blurred.flatten()
blurred_mobility = blurred_mobility.T

In [ ]:
x = [Parameters.OUTBREAK_FIRST_DAY + datetime.timedelta(days=j) for j in range(1072)]
plt.figure(figsize=(15,4))
plt.plot(x, blurred_mobility)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
dates = []

In [ ]:
out_df = pd.DataFrame(blurred_mobility[ (Ontario['date'].min() - Parameters.OUTBREAK_FIRST_DAY).days:(Ontario['date'].max() - Parameters.OUTBREAK_FIRST_DAY).days].astype(int), columns=['retail', 'grocery', 'park', 'trainsit', 'workplace', 'residential'])
print(out_df)

In [ ]:
out_df['date'] =  [Ontario['date'].min()+ datetime.timedelta(days=j) for j in range((Ontario['date'].max()  - Ontario['date'].min()).days)]
out_df.to_csv(os.getcwd()[:-5] + 'Model Dependencies/' + 'blurred_average_mobility.csv')